In [1]:
from enum import Enum

class DetailChannelType(Enum):
    def __init__(self, code, detail_channel_type_name):
        self.code = code
        self.detail_channel_type_name = detail_channel_type_name
        
    LECTURE = ("LECTURE", "강의채널")
    MISSION = ("MISSION", "미션채널")
    PHOTOVIEW = ("PHOTOVIEW", "포토뷰채널")
    NORMAL = ("NORMAL", "일반채널")
    NOTICE = ("NOTICE", "공지사항채널")
    QNA = ("QNA", "질문채널")
    
    def eqauls(self, code):
        return self.code == code

In [85]:
import boto3
from boto3.dynamodb.conditions import Key, Attr

dyn_resource = boto3.resource('dynamodb')
channel_table = dyn_resource.Table("channel-dev_cta_20221225-classu_ted")
post_table = dyn_resource.Table("post-dev_cta_20221225-classu_ted")

YES = "Y"
NO = "N"

def make_begins_with_query(pk_value, sk_value):
    query = {
        "KeyConditionExpression" : Key("partition_key").eq(pk_value) & Key("sort_key").begins_with(sk_value)
    }
    return query


def make_begins_with_query_on_index(index_name, pk, pk_value, sk, sk_value):
    query = {
        "KeyConditionExpression" : Key(pk).eq(pk_value) & Key(sk).begins_with(sk_value),
        "IndexName" : index_name
    }
    return query

In [94]:
class ChannelService:
    def __init__(self, channel_id):
        self.channel_id = channel_id
        
        
    def get_all_zero_level_post(self, sort_by, is_teacher, user_id, channel_item):
        detail_channel_type = channel_item["detail_channel_type"]
        
        if DetailChannelType.LECTURE.eqauls(detail_channel_type):
            return self.get_lecture_channel_zero_level_post_list(is_teacher, user_id)
        else:
            return
    
            
    def get_lecture_channel_zero_level_post_list(self, is_teacher, user_id):
        query_result = self.query_video_channel_zero_level_post_list()

        if not query_result:
            return []
            

        post_id_userview_dict = {}
        for i in self.query_userview_list(user_id):
            post_id_userview_dict[i["post_id"]] = i
            
        if is_teacher:
            return [ self.make_lecture_post_for_teacher(i, post_id_userview_dict) for i in query_result ]

        mission_completed_post_id_list = [ i["sort_key"].split("#")[2].split("post@")[1] for i in query_mission_completed_post_list(user_id) ]
        return [ self.make_lecture_post_for_student(i, mission_completed_post_id_list, post_id_userview_dict) for i in query_result ]
    
    
    def query_video_channel_zero_level_post_list(self):
        query = make_begins_with_query_on_index("channel_id-sk_post_post_order-index", "channel_id", self.channel_id, "sk_post_post_order", "ACTIVE")
        return post_table.query(**query)["Items"]
    
    
    def query_userview_list(self, user_id):
        query = make_begins_with_query(f"user@{user_id}", f"user_view#channel@{self.channel_id}")
        return post_table.query(**query)["Items"]


    def query_mission_completed_post_list(self, user_id):
        query = make_begins_with_query(f"user@{user_id}", f"mission#channel@{self.channel_id}")
        return post_table.query(**query)["Items"]


    def make_lecture_post_for_teacher(self, item, post_id_userview_dict):
        post_id = item["post_id"]
        lecture_file_info = item["video_file_info_list"][0]

        if post_id in post_id_userview_dict:
            userview_item = post_id_userview_dict.get(post_id)
            post_info = {
                "post_id": post_id,
                "post_name": item["post_name"],
                "thumbnail_file_info_list": item["thumbnail_file_info_list"],
                "reply_post_count": str(datetime.timedelta(milliseconds=int(item["reply_post_count"]))),
                "view_count": str(datetime.timedelta(milliseconds=int(item["view_count"]))),
                "mission_yn": YES if item.get("mission_contents") or item.get("mission_media_file_info_list") else NO,
                "running_time": lecture_file_info["running_time"],
                "user_view_yn": YES,
                "user_view_time_record": str(datetime.timedelta(milliseconds=int(userview_item["user_view_time_record"]))),
                "delete_yn": item["delete_yn"]
            }

        else:
            post_info = {
                "post_id": post_id,
                "post_name": item["post_name"],
                "thumbnail_file_info_list": item["thumbnail_file_info_list"],
                "reply_post_count": str(datetime.timedelta(milliseconds=int(item["reply_post_count"]))),
                "view_count": str(datetime.timedelta(milliseconds=int(item["view_count"]))),
                "mission_yn": YES if item.get("mission_contents") or item.get("mission_media_file_info_list") else NO,
                "running_time": lecture_file_info["running_time"],
                "user_view_yn": NO,
                "delete_yn": item["delete_yn"]
            }

        return post_info


    def make_lecture_post_for_student(self, item, mission_completed_post_id_list, post_id_userview_dict):
        post_id = item["post_id"]
        userview_item = post_id_userview_dict[post_id]
        is_viewed = post_id in post_id_userview_dict

        if is_viewed:
            post_id_userview_item = post_id_userview_dict[post_id]
            post_info = {
                "post_id": post_id,
                "post_name": item["post_name"],
                "thumbnail_file_info_list": item["thumbnail_file_info_list"],
                "reply_post_count": str(datetime.timedelta(milliseconds=int(item["reply_post_count"]))),
                "view_count": str(datetime.timedelta(milliseconds=int(item["view_count"]))),
                "mission_complete_yn": YES if post_id in mission_completed_post_id_list else NO,
                "running_time": item["running_time"],
                "user_view_yn": YES,
                "user_view_complete_yn": userview_item["user_view_complete_yn"],
                "user_view_time_record": str(datetime.timedelta(milliseconds=int(userview_item["user_view_time_record"]))),
                "delete_yn": item["delete_yn"]
            }

        else:
            post_info = {
                "post_id": post_id,
                "post_name": item["post_name"],
                "thumbnail_file_info_list": item["thumbnail_file_info_list"],
                "reply_post_count": str(datetime.timedelta(milliseconds=int(item["reply_post_count"]))),
                "view_count": str(datetime.timedelta(milliseconds=int(item["view_count"]))),
                "mission_complete_yn": YES if post_id in mission_completed_post_id_list else NO,
                "running_time": item["running_time"],
                "user_view_yn": NO,
                "delete_yn": item["delete_yn"]
            }

        return post_info

In [95]:
import json

channel_item = channel_table.get_item(Key={"partition_key": "channel@600fa200-4090-44a6-94b9-6a9c7da92568", "sort_key": "channel"})["Item"]

result = ChannelService("600fa200-4090-44a6-94b9-6a9c7da92568").get_all_zero_level_post("NONE", True, "1", channel_item)
print(json.dumps(result))

[{"post_id": "f906533a-4e1f-4589-845b-1cc7e9222463", "post_name": "\uc6c3\uc74c\ucc38\uae30 \ucc4c\ub9b0\uc9c0 #1", "thumbnail_file_info_list": [{"file_url": "https://thumb.midibus.kinxcdn.com/346/185a81b997baa872.png", "file_type": "IMAGE"}], "reply_post_count": "0:00:00", "view_count": "0:00:01.137000", "mission_yn": "Y", "running_time": "00:04:04", "user_view_yn": "N", "delete_yn": "N"}, {"post_id": "59eaea7f-203d-49ff-ab0e-1c4d459d4e09", "post_name": "\uc6c3\uc74c\ucc38\uae30 \ucc4c\ub9b0\uc9c0 #2", "thumbnail_file_info_list": [{"file_url": "https://thumb.midibus.kinxcdn.com/346/185a31baa1bb73e9.png", "file_type": "IMAGE"}], "reply_post_count": "0:00:00", "view_count": "0:00:00", "mission_yn": "Y", "running_time": "00:10:13", "user_view_yn": "N", "delete_yn": "N"}, {"post_id": "e5e5100d-169e-4136-9af1-8db959aa1147", "post_name": "\uc6c3\uc74c\ucc38\uae30 \ucc4c\ub9b0\uc9c0 #3", "thumbnail_file_info_list": [{"file_url": "https://thumb.midibus.kinxcdn.com/346/185a4d45569a5564.png", "